# Numerical Calcs
## There will be fun


![alt-text](img/linearField.png "Linearized Field")
https://en.wikipedia.org/wiki/Linearized_gravity



![alt-text](img/zfpTheory.png "zfp operates on blocks")
https://zfp.readthedocs.io/en/release0.5.5/overview.html


16.0


In [6]:
# number of points per block
N = 128
bitsPerValue = 1
componentCount = 10
MB = N**4*bitsPerValue*componentCount/8/1e6
print('MB per block: ' + str(MB))

#surface area = N**3*8 (teseract surface area). 

#percent of blocks on surface
fractionOnSurface = (N**3*8)/N**4
print('fraction of points on surface of block: ' + str(fractionOnSurface))

# time to do block:
onePointFPInstructions = 5000
speed = 3e9
totalInstructions = onePointFPInstructions*N**4
blockTime = totalInstructions/speed
print('time for a block: ' + str(blockTime) + ' seconds.')



# time to do entire run :
M = 10
blocks = M**4
concurrency = 4
totalTimePerSweep = blocks*blockTime/concurrency
print('time for a sweep of ' + str(M*N) + ' grid  of ' + str(M**4) + ' blocks is ' + str(totalTimePerSweep/86400) + ' days.')

GB = (M*N)**4*bitsPerValue*componentCount/8/1e9
print('Grid is ' + str(M*N) + '^4, so total storage is ' + str(GB/1000) + 'TB, memory when running is ' + str(concurrency*MB/1000) + 'GB')

# time to use seperate files for each block, a folder for each revision, and such. 
# driver process uses completed files

# load into memory, 1GB/sec 


MB per block: 335.54432
fraction of points on surface of block: 0.0625
time for a block: 447.39242666666667 seconds.
time for a sweep of 1280 grid  of 10000 blocks is 12.945382716049382 days.
Grid is 1280^4, so total storage is 3.3554432000000003TB, memory when running is 1.34217728GB


### plan is to have each block have its own arrays. Each block needs to be big eough so that the zfp can do its magic, and surface areas need to be cut so that overlaps on surfaces can work. 

In [24]:
# multires
# imagine (draw) a cube with a black hole at centre, 
# in 3D, ignoring time for now, we have for 3 levels of multires
# 4^3 is 64 so for 
# Large 64 - 8 taken in central to be subdivided =  56
# Those 8 are made into a 4x4x4 block of 
# Mediums - 64, but 8 taken for small blocks
# Smalls 64 (we stop there)
# so (4^4 - 8)*(L-1) + 4^4 blocks
# central 8 are 256 on a side 
# all others are 128 on a side
L = 5
n128 = (4**3 - 8)*(L - 1) + 4**4 - 8
# so 
n128 = (4**3 - 8)*L
n256 = 8
print('total normal blocks in one time slice: ' + str(n128) + ' total hig res blocks in one time slice: ' + str(n256) + '.')

# how long we run the simulation for. Nt = 5 means that the time axis is 5x longer than the space axis. 
Nt = 5 
# number of blocks in time direction is simply a multiplier on that
n128t = (4**4 - 8)*L*Nt
n256t = 8*Nt
print('total normal blocks: ' + str(n128t) + ' total high res blocks: ' + str(n256t) + '.')

# so 
blockTime128 = onePointFPInstructions*128**4/speed
print('time for a normal block: ' + str(blockTime128) + ' seconds.')

blockTime256 = onePointFPInstructions*256**4/speed
print('time for a hi res block: ' + str(blockTime256) + ' seconds.')

#total simulation time for one sweep:
print('Total simulation time per sweep: ' + str(blockTime256) + ' seconds.')


concurrency = 50
totalTimePerSweep = n128t*blockTime128/concurrency + n256t*blockTime256/concurrency
print('time for a sweep of grid  of ' + str(n128t) + ' normal and ' + str(n256t) + ' high res blocks is ' + str(totalTimePerSweep/86400) + ' days.')

sweeps = 30
print('Total simulation time '  + str(sweeps*totalTimePerSweep/86400) + ' days.')


# Note that the time is obvisouly a guess, but in addition, the first sweep would be over a 256x256x256x256*Nt block, which would 
# subdivide into the smaller ones after a few sweeps. This helps with getting the solution approximated quickly across the grid





total normal blocks in one time slice: 280 total hig res blocks in one time slice: 8.
total normal blocks: 6200 total high res blocks: 40.
time for a normal block: 447.39242666666667 seconds.
time for a hi res block: 7158.278826666667 seconds.
Total simulation time per sweep: 7158.278826666667 seconds.
time for a sweep of grid  of 6200 normal and 40 high res blocks is 0.7083713422222222 days.
Total simulation time 21.251140266666667 days.
